# (Almost) pure functional solution to advent of code, day 7 (part 1) ([link])

An attempt to create pure functional solution to the problem. There are no for-loops and limited mutations of data structures (i.e., all functions are pure)

[1]:https://adventofcode.com/2018/day/7

In [1]:
import re
from functools import partial
from toolz.curried import flip, map
from toolz import pipe
from typing import List, Text, Iterator, Tuple, Set, Sequence

# Helper Types
Task = Text
Pair = Tuple[Task, Task]
TaskDeps = Tuple[Task, Set[Task]]

In [2]:
def match(lines: List[Text]) ->Iterator[Pair]:
    """Parses the input lines into pairs of dependencies."""
    rgx = "Step ([A-Z]) must be finished before step ([A-Z]) can begin."
    def match_expr(txt, rgx):
        return re.match(rgx, txt).groups()
    match_inner = partial(match_expr, rgx=rgx)
    return map(match_inner, lines)

In [3]:
def create_deps(deps: Iterator[Pair]) -> Tuple[TaskDeps]:
    """For each task, returns all its dependencies."""
    def deps_dict_inner(iterable, current_dict):
        new_dict = {k:v for k, v in current_dict.items()}
        try:
            head, tail = next(iterable), iterable
        except StopIteration:
            return new_dict
        else:
            dependency, task = head
            if task in new_dict:
                new_dict[task].add(dependency)
            else:
                new_dict[task] = set()
                new_dict[task].add(dependency)
            if dependency not in new_dict:
                new_dict[dependency] = set()
            return deps_dict_inner(tail, new_dict)
    dd = deps_dict_inner(deps, {})
    return tuple(dd.items())

In [4]:
def get_remaining_dependencies(tasks: Iterator[TaskDeps], 
                               completed_tasks: List[Task]) -> Iterator[TaskDeps]:
    """Given a list of completed tasks and remaining tasks, return the remaining
    dependency for each task."""
    def get_remaining_dependencies_inner(task_dependencies):
        task, dependencies = task_dependencies
        return task, list(filter(lambda y: y not in completed_tasks, dependencies))
    return map(get_remaining_dependencies_inner, tasks)
    
def get_next_task(tasks: Iterator[TaskDeps], completed_tasks: List[Task]) -> Task:
    """Get the next task to be worked on (the ones with all dependencies resolved)."""
    return sorted(filter(lambda task_deps : (len(task_deps[1]) == 0 and 
                                             task_deps[0] not in completed_tasks),
                         tasks),
                  key=lambda task_deps: task_deps[0])[0][0]

def complete_next_task(next_task: Task,
                       completed_tasks: List[Task], 
                       tasks: Iterator[Task]) -> Tuple[List[Task], List[TaskDeps]]:
    """Bookkeeping: Add the task to the completed task and remove it from the remaining tasks."""
    return completed_tasks + [next_task], [i for i in tasks if i[0] != next_task]


def get_order(tasks: Task) -> List[Task]:
    """Find the order in which we should work on the tasks."""
    def get_order_inner(remaining_tasks, completed_tasks):
        if remaining_tasks == []:
            return completed_tasks
        
        p_complete_next_task = partial(complete_next_task, 
                                       completed_tasks=completed_tasks, 
                                       tasks=remaining_tasks)
        p_get_remaining_dependencies = partial(get_remaining_dependencies, 
                                              completed_tasks=completed_tasks)
        p_get_next_task = partial(get_next_task, 
                                  completed_tasks=completed_tasks)
        completed_tasks, remaining_tasks = pipe(remaining_tasks, 
                                                p_get_remaining_dependencies,
                                                p_get_next_task,
                                                p_complete_next_task)
        return get_order_inner(remaining_tasks, completed_tasks)
    return get_order_inner(tasks, [])

In [5]:
puzzle = """Step Q must be finished before step O can begin.
Step Z must be finished before step G can begin.
Step W must be finished before step V can begin.
Step C must be finished before step X can begin.
Step O must be finished before step E can begin.
Step K must be finished before step N can begin.
Step P must be finished before step I can begin.
Step X must be finished before step D can begin.
Step N must be finished before step E can begin.
Step F must be finished before step A can begin.
Step U must be finished before step Y can begin.
Step M must be finished before step H can begin.
Step J must be finished before step B can begin.
Step B must be finished before step E can begin.
Step S must be finished before step L can begin.
Step A must be finished before step L can begin.
Step E must be finished before step L can begin.
Step L must be finished before step G can begin.
Step D must be finished before step I can begin.
Step Y must be finished before step I can begin.
Step I must be finished before step G can begin.
Step G must be finished before step R can begin.
Step V must be finished before step T can begin.
Step R must be finished before step H can begin.
Step H must be finished before step T can begin.
Step S must be finished before step E can begin.
Step C must be finished before step E can begin.
Step P must be finished before step T can begin.
Step I must be finished before step H can begin.
Step O must be finished before step P can begin.
Step M must be finished before step L can begin.
Step S must be finished before step D can begin.
Step P must be finished before step D can begin.
Step P must be finished before step R can begin.
Step I must be finished before step R can begin.
Step Y must be finished before step G can begin.
Step Q must be finished before step L can begin.
Step N must be finished before step R can begin.
Step J must be finished before step E can begin.
Step N must be finished before step T can begin.
Step B must be finished before step V can begin.
Step Q must be finished before step B can begin.
Step J must be finished before step H can begin.
Step F must be finished before step B can begin.
Step W must be finished before step X can begin.
Step S must be finished before step T can begin.
Step J must be finished before step G can begin.
Step O must be finished before step R can begin.
Step K must be finished before step B can begin.
Step Z must be finished before step O can begin.
Step Q must be finished before step S can begin.
Step K must be finished before step V can begin.
Step B must be finished before step R can begin.
Step J must be finished before step T can begin.
Step E must be finished before step T can begin.
Step G must be finished before step V can begin.
Step D must be finished before step Y can begin.
Step M must be finished before step Y can begin.
Step F must be finished before step G can begin.
Step C must be finished before step P can begin.
Step V must be finished before step R can begin.
Step R must be finished before step T can begin.
Step J must be finished before step Y can begin.
Step U must be finished before step R can begin.
Step Z must be finished before step F can begin.
Step Q must be finished before step V can begin.
Step U must be finished before step M can begin.
Step J must be finished before step R can begin.
Step L must be finished before step V can begin.
Step W must be finished before step K can begin.
Step B must be finished before step Y can begin.
Step O must be finished before step N can begin.
Step D must be finished before step V can begin.
Step P must be finished before step B can begin.
Step U must be finished before step I can begin.
Step O must be finished before step T can begin.
Step S must be finished before step G can begin.
Step X must be finished before step A can begin.
Step U must be finished before step T can begin.
Step A must be finished before step I can begin.
Step B must be finished before step G can begin.
Step N must be finished before step Y can begin.
Step Z must be finished before step J can begin.
Step M must be finished before step D can begin.
Step U must be finished before step A can begin.
Step S must be finished before step R can begin.
Step Z must be finished before step A can begin.
Step Y must be finished before step R can begin.
Step E must be finished before step Y can begin.
Step N must be finished before step G can begin.
Step Z must be finished before step X can begin.
Step P must be finished before step X can begin.
Step Z must be finished before step T can begin.
Step Z must be finished before step P can begin.
Step V must be finished before step H can begin.
Step P must be finished before step L can begin.
Step L must be finished before step H can begin.
Step X must be finished before step V can begin.
Step W must be finished before step G can begin.
Step N must be finished before step D can begin.
Step Z must be finished before step U can begin."""

In [6]:
ret = pipe(puzzle.split('\n'), match, create_deps, get_order)
"".join(ret)

'CQSWKZFJONPBEUMXADLYIGVRHT'